In [ ]:
%load_ext autoreload
%autoreload 2

import sys
import os
import requests
sys.path.append(os.path.abspath('../src'))

from load_data import load_raw
from preprocess import clean_data, select_column, rename, sales_and_residential, drop_na, add_cities, make_zero
from train_model import train_random_forest
from evaluate import evaluate_model, compute_shap_local, compute_shap_global


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Step 1: Load data and delete the data that is not our target cities

In [2]:
import pandas as pd

path = r"C:\Users\ktlee\Downloads\all_transactions.csv"
df_raw = load_raw(path)



c:\Users\ktlee\20251103_comp3314\COMP3314-group-25\src\load_data.py:6: DtypeWarning: Columns (14,18,23,24,26,28,30,31,32,33,41) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path)


Index(['Identifiant de document', 'Reference document', '1 Articles CGI',
       '2 Articles CGI', '3 Articles CGI', '4 Articles CGI', '5 Articles CGI',
       'No disposition', 'Date mutation', 'Nature mutation', 'Valeur fonciere',
       'No voie', 'B/T/Q', 'Type de voie', 'Code voie', 'Voie', 'Code postal',
       'Commune', 'Code departement', 'Code commune', 'Prefixe de section',
       'Section', 'No plan', 'No Volume', '1er lot',
       'Surface Carrez du 1er lot', '2eme lot', 'Surface Carrez du 2eme lot',
       '3eme lot', 'Surface Carrez du 3eme lot', '4eme lot',
       'Surface Carrez du 4eme lot', '5eme lot', 'Surface Carrez du 5eme lot',
       'Nombre de lots', 'Code type local', 'Type local', 'Identifiant local',
       'Surface reelle bati', 'Nombre pieces principales', 'Nature culture',
       'Nature culture speciale', 'Surface terrain'],
      dtype='object')
<bound method DataFrame.count of          Identifiant de document  Reference document  1 Articles CGI  \
0   

In [8]:
df_renamed = rename(df_raw)
df_selected = select_column(df_renamed)
df_filtered = sales_and_residential(df_selected)
df_dropped = drop_na(df_filtered)
df_with_city = add_cities(df_dropped)
df_with_zero = make_zero(df_with_city)

cycle = 500
if False:
    for i in range(0, len(df_with_zero), cycle):
        df_i = df.iloc[i:i+cycle]
        result = 0 

c:\Users\ktlee\20251103_comp3314\COMP3314-group-25\src\preprocess.py:88: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['City'] = df['Postal code'].apply(assign_city)
<string>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [9]:
print(df_with_zero.head)
#df_dropped.to_csv("comp3314_semi_clean_csv")

<bound method NDFrame.head of          Document ID Date of mutation Nature of mutation Property value  \
107553           NaN       28/02/2023              Vente       80000,00   
111739           NaN       12/06/2023              Vente      135000,00   
112798           NaN       07/07/2023              Vente       55000,00   
113783           NaN       21/07/2023              Vente       25000,00   
115624           NaN       15/09/2023              Vente      285000,00   
...              ...              ...                ...            ...   
8688537          NaN       23/06/2025              Vente      835000,00   
8688539          NaN       25/06/2025              Vente      375000,00   
8688540          NaN       25/06/2025              Vente     1370000,00   
8688548          NaN       27/06/2025              Vente      550000,00   
8688549          NaN       27/06/2025              Vente      550386,10   

         Street number        Street name  Postal code  Municipality 

Step 2: For each city, pre-process and train

In [ ]:
df = df_with_city
cities = df['City'].unique()

metrics_table = dict()

for city in cities:
    df_city = df[df['City'] == city].copy()
    X, y = clean_data(df_city)

    case_model = 1 #<-- Change this

    #Case 1: Let the program to find best model
    if case_model == 1:
        model, params, X_test, y_test = train_random_forest(X, y)

    #Case 2: Self define the model
    if case_model == 2:
        param_grid = {
                'n_estimators': [2500],
                'max_depth': [32],
                'bootstrap': [False]
            }
        model, params, X_test, y_test = train_random_forest(X, y, param_grid)


    metrics = evaluate_model(model, X_test, y_test)
    metrics_table[city] = metrics
    print(f"City {city}'s model training is done.")

Step 4: SHAP analysis

In [ ]:
import shap
explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(X)
shap.summary_plot(shap_values, X)